In [1]:
import os
import cv2
import numpy as np
import h5py
import keras
from PIL import Image
from matplotlib import pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.applications.vgg16 import VGG16
from keras.utils import to_categorical
from keras.models import Model,Sequential
from keras.layers import *

In [2]:
emotion_labels=["Angry","Disgust","Fear","Happy","Sad","Surprise","Neutral"]

In [3]:
num_classes=len(emotion_labels)

In [4]:
fer_data=h5py.File('C:/Users/Student/Desktop/Facial_Emotion_Detection/FER-2013/data.h5','r',driver='core')
print(fer_data)

<HDF5 file "data.h5" (mode r)>


In [5]:
fer_x=np.asarray(fer_data['Training_pixel']).reshape(28709,48,48,1)

In [6]:
fer_y=fer_data['Training_label']

In [7]:
X_train=fer_x

In [8]:
X_test_public=np.asarray(fer_data['PublicTest_pixel']).reshape((3589,48,48,1))

In [9]:
X_test_private=np.asarray(fer_data['PrivateTest_pixel']).reshape((3589,48,48,1))

In [10]:
num_train=X_train.shape[0]

In [11]:
num_val_public=X_test_public.shape[0]

In [12]:
num_val_private=X_test_private.shape[0]

In [13]:
print('Training:',X_train.shape)

Training: (28709, 48, 48, 1)


In [14]:
print('PublicTest:',X_test_public.shape)

PublicTest: (3589, 48, 48, 1)


In [15]:
print('PrivateTest:',X_test_private.shape)

PrivateTest: (3589, 48, 48, 1)


In [16]:
Y_train=fer_data['Training_label']

In [17]:
Y_train=to_categorical(Y_train,num_classes)

In [18]:
Y_test_public=fer_data['PublicTest_label']

In [19]:
Y_test_private=fer_data['PrivateTest_label']

In [20]:
Y_test_public=to_categorical(Y_test_public,num_classes)

In [21]:
Y_test_private=to_categorical(Y_test_private,num_classes)

In [22]:
print('Training:',Y_train.shape)

Training: (28709, 7)


In [23]:
print('PublicTest:',Y_test_public.shape)

PublicTest: (3589, 7)


In [24]:
print('PrivateTest:',Y_test_private.shape)

PrivateTest: (3589, 7)


In [25]:
from keras.layers import Convolution2D,Activation,BatchNormalization,MaxPooling2D,Dropout,Dense,Flatten,AveragePooling2D

In [26]:
from keras.models import Sequential

In [27]:
def get_cnn_model():
    model=Sequential()
    
    model.add(Convolution2D(64,(3,1),padding='same',input_shape=(48,48,1)))
    model.add(Convolution2D(64,(1,3),padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2),strides=None,padding='same'))
    model.add(Dropout(0.25))
    
    model.add(Convolution2D(128,(3,1),padding='same'))
    model.add(Convolution2D(128,(1,3),padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2),strides=None,padding='same'))
    model.add(Dropout(0.25))
    
    model.add(Convolution2D(256,(3,1),padding='same'))
    model.add(Convolution2D(256,(1,3),padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2),strides=None,padding='same'))
    model.add(Dropout(0.25))
    
    model.add(Convolution2D(512,(3,1),padding='same'))
    model.add(Convolution2D(512,(1,3),padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2),strides=None,padding='same'))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    
    model.add(Dense(512))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.25))
    
    model.add(Dense(7))
    model.add(Activation('softmax'))
    return model

In [28]:
cnn_model=get_cnn_model()

In [29]:
cnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 64)        256       
                                                                 
 conv2d_1 (Conv2D)           (None, 48, 48, 64)        12352     
                                                                 
 batch_normalization (BatchN  (None, 48, 48, 64)       256       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 48, 48, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 24, 24, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 24, 24, 64)        0

In [30]:
from keras.preprocessing.image import ImageDataGenerator

In [31]:
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau

In [32]:
from keras import backend as K

In [33]:
def fbeta(y_true, y_pred, threshold_shift=0):
    beta = 1
    y_pred = K.clip(y_pred, 0, 1)
    y_pred_bin = K.round(y_pred + threshold_shift)

    tp = K.sum(K.round(y_true * y_pred_bin), axis=1) + K.epsilon()
    fp = K.sum(K.round(K.clip(y_pred_bin - y_true, 0, 1)), axis=1)
    fn = K.sum(K.round(K.clip(y_true - y_pred, 0, 1)), axis=1)

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    beta_squared = beta ** 2
    return K.mean((beta_squared + 1) * (precision * recall) / (beta_squared * precision + recall + K.epsilon()))

In [34]:
def train_cnn_model(model):
    filepath='../opt/fer-cnn/Model.{epoch:02d}-{val_acc:.4f}.hdf5'
    checkpointer=ModelCheckpoint(filepath,monitor='val_loss',verbose=1,save_best_only=False,mode='auto')
    reduce_lr=ReduceLROnPlateau(monitor='val_loss',factor=0.1,patience=10,verbose=0,mode='auto',cooldown=0,min_lr=0)
    early_stop=EarlyStopping(monitor='val_loss',min_delta=0,patience=0,verbose=0,mode='auto')
    datagen=ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        rotation_range=0,
        width_shift_range=0.0,
        height_shift_range=0.0,
        horizontal_flip=True,
        vertical_flip=False,
    )
    
    datagen.fit(X_train)
    datagen.fit(X_test_public)
    datagen.fit(X_test_private)
    batch_size=32
    
    num_epochs=200
    model.compile(loss='categorical_crossentropy',
                      optimizer='adam',
                 metrics=[fbeta,'acc'])
    train_flow=datagen.flow(X_train,Y_train,batch_size=batch_size)
    test_flow_public=datagen.flow(X_test_public,Y_test_public)
    test_flow_private=datagen.flow(X_test_private,Y_test_private)
    
    history_public = model.fit_generator(train_flow,
                    steps_per_epoch=len(X_train) / batch_size,
                    epochs=num_epochs, 
                    verbose=1, 
                    validation_data=test_flow_public, 
                    validation_steps=len(X_test_public) / batch_size,
                    callbacks=[checkpointer, reduce_lr, checkpointer])
    history_private = model.fit_generator(train_flow,
                    steps_per_epoch=len(X_train) / batch_size,
                    epochs=num_epochs, 
                    verbose=1, 
                    validation_data=test_flow_private, 
                    validation_steps=len(X_test_private) / batch_size,
                    callbacks=[checkpointer, reduce_lr, checkpointer])
    return history_public,history_private,model

In [ ]:
history_public,history_private,cnn_model = train_cnn_model(cnn_model)

C:\Users\Student\AppData\Local\Temp\ipykernel_11944\152893961.py:32: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history_public = model.fit_generator(train_flow,


Epoch 1/200
898/897 [==============================] - ETA: 0s - loss: 1.6745 - fbeta: 0.1691 - acc: 0.3676
Epoch 1: saving model to ../opt/fer-cnn\Model.01-0.4536.hdf5

Epoch 1: saving model to ../opt/fer-cnn\Model.01-0.4536.hdf5
897/897 [==============================] - 444s 490ms/step - loss: 1.6745 - fbeta: 0.1691 - acc: 0.3676 - val_loss: 1.4248 - val_fbeta: 0.2898 - val_acc: 0.4536 - lr: 0.0010
Epoch 2/200
898/897 [==============================] - ETA: 0s - loss: 1.3637 - fbeta: 0.2786 - acc: 0.4813
Epoch 2: saving model to ../opt/fer-cnn\Model.02-0.4622.hdf5

Epoch 2: saving model to ../opt/fer-cnn\Model.02-0.4622.hdf5
897/897 [==============================] - 506s 564ms/step - loss: 1.3637 - fbeta: 0.2786 - acc: 0.4813 - val_loss: 1.4187 - val_fbeta: 0.3482 - val_acc: 0.4622 - lr: 0.0010
Epoch 3/200
898/897 [==============================] - ETA: 0s - loss: 1.2640 - fbeta: 0.3166 - acc: 0.5200
Epoch 3: saving model to ../opt/fer-cnn\Model.03-0.4968.hdf5

Epoch 3: saving mode

898/897 [==============================] - ETA: 0s - loss: 0.8583 - fbeta: 0.5619 - acc: 0.6770
Epoch 21: saving model to ../opt/fer-cnn\Model.21-0.6007.hdf5

Epoch 21: saving model to ../opt/fer-cnn\Model.21-0.6007.hdf5
897/897 [==============================] - 479s 534ms/step - loss: 0.8583 - fbeta: 0.5619 - acc: 0.6770 - val_loss: 1.0620 - val_fbeta: 0.4879 - val_acc: 0.6007 - lr: 0.0010
Epoch 22/200
898/897 [==============================] - ETA: 0s - loss: 0.8381 - fbeta: 0.5739 - acc: 0.6843
Epoch 22: saving model to ../opt/fer-cnn\Model.22-0.6311.hdf5

Epoch 22: saving model to ../opt/fer-cnn\Model.22-0.6311.hdf5
897/897 [==============================] - 473s 528ms/step - loss: 0.8381 - fbeta: 0.5739 - acc: 0.6843 - val_loss: 1.0309 - val_fbeta: 0.5541 - val_acc: 0.6311 - lr: 0.0010
Epoch 23/200
898/897 [==============================] - ETA: 0s - loss: 0.8277 - fbeta: 0.5822 - acc: 0.6923
Epoch 23: saving model to ../opt/fer-cnn\Model.23-0.6057.hdf5

Epoch 23: saving model to

898/897 [==============================] - ETA: 0s - loss: 0.5589 - fbeta: 0.7331 - acc: 0.7952
Epoch 41: saving model to ../opt/fer-cnn\Model.41-0.6590.hdf5

Epoch 41: saving model to ../opt/fer-cnn\Model.41-0.6590.hdf5
897/897 [==============================] - 483s 538ms/step - loss: 0.5589 - fbeta: 0.7331 - acc: 0.7952 - val_loss: 1.0031 - val_fbeta: 0.6015 - val_acc: 0.6590 - lr: 1.0000e-05
Epoch 42/200
898/897 [==============================] - ETA: 0s - loss: 0.5686 - fbeta: 0.7281 - acc: 0.7919
Epoch 42: saving model to ../opt/fer-cnn\Model.42-0.6548.hdf5

Epoch 42: saving model to ../opt/fer-cnn\Model.42-0.6548.hdf5
897/897 [==============================] - 455s 507ms/step - loss: 0.5686 - fbeta: 0.7281 - acc: 0.7919 - val_loss: 1.0102 - val_fbeta: 0.5998 - val_acc: 0.6548 - lr: 1.0000e-05
Epoch 43/200
898/897 [==============================] - ETA: 0s - loss: 0.5664 - fbeta: 0.7257 - acc: 0.7925
Epoch 43: saving model to ../opt/fer-cnn\Model.43-0.6584.hdf5

Epoch 43: saving 

Epoch 61/200
898/897 [==============================] - ETA: 0s - loss: 0.5494 - fbeta: 0.7368 - acc: 0.7993
Epoch 61: saving model to ../opt/fer-cnn\Model.61-0.6578.hdf5

Epoch 61: saving model to ../opt/fer-cnn\Model.61-0.6578.hdf5
897/897 [==============================] - 435s 485ms/step - loss: 0.5494 - fbeta: 0.7368 - acc: 0.7993 - val_loss: 1.0048 - val_fbeta: 0.6001 - val_acc: 0.6578 - lr: 1.0000e-07
Epoch 62/200
898/897 [==============================] - ETA: 0s - loss: 0.5500 - fbeta: 0.7359 - acc: 0.7985
Epoch 62: saving model to ../opt/fer-cnn\Model.62-0.6565.hdf5

Epoch 62: saving model to ../opt/fer-cnn\Model.62-0.6565.hdf5
897/897 [==============================] - 440s 490ms/step - loss: 0.5500 - fbeta: 0.7359 - acc: 0.7985 - val_loss: 1.0047 - val_fbeta: 0.6010 - val_acc: 0.6565 - lr: 1.0000e-07
Epoch 63/200
898/897 [==============================] - ETA: 0s - loss: 0.5591 - fbeta: 0.7328 - acc: 0.7950
Epoch 63: saving model to ../opt/fer-cnn\Model.63-0.6578.hdf5

Epoc

Epoch 81/200
 20/897 [..............................] - ETA: 15:28 - loss: 0.5033 - fbeta: 0.7688 - acc: 0.8359

In [ ]:
def eval_model_public(history,model):
    batch_size=32
    score=model.evaluate(X_test_public,Y_test_public,steps=(int)(len(X_test_public)/batch_size))
    print('Evaluation loss:',score[0])
    print('Evaluation accuracy:',score[1])
                        
    plt.plot(history.history['acc'],color='b',label='Training')
    plt.plot(history.history['val_acc'],color='g',label='Validation')
    plt.title('Accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(loc='upper left')
    plt.show()
                         
    plt.plot(history.history['loss'],color='b',label='Training')
    plt.plot(history.history['val_loss'],color='g',label='Validation')
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(loc='lower left')
    plt.show()
                         
    y_pred=model.predict_classes(X_test_public)
    y_true=np.asarray([np.argmax(i) for i in Y_test_public])
    
    from sklearn.metrics import confusion_matrix
    import seaborn as sns
    
    cm=confusion_matrix(y_true,y_pred)
    cm_normalised=cm.astype('float')/cm.sum(axis=1)[:,np.newaxis]
    sns.set(font_scale=1.5)
    fig, ax = plt.subplots(figsize=(10,10))
    ax=sns.heatmap(cm_normalised,
                  annot=True,
                  linewidths=0,
                  square=False,
                  cmap="Greens",
                  yticklabels=emotion_labels,
                  xticklabels=emotion_labels,
                  vmin=0,
                  vmax=np.max(cm_normalised),
                  fmt=".2f",
                  annot_kws={"size":20})
    ax.set(xlabel='Predicted label',ylabel='True label')

In [ ]:
def eval_model_private(history,model):
    batch_size=32
    score=model.evaluate(X_test_private,Y_test_private,steps=(int)(len(X_test_private)/batch_size))
    print('Evaluation loss:',score[0])
    print('Evaluation accuracy:',score[1])
                        
    plt.plot(history.history['acc'],color='b',label='Training')
    plt.plot(history.history['val_acc'],color='g',label='Validation')
    plt.title('Accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(loc='upper left')
    plt.show()
                         
    plt.plot(history.history['loss'],color='b',label='Training')
    plt.plot(history.history['val_loss'],color='g',label='Validation')
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(loc='lower left')
    plt.show()
    

  

In [ ]:
eval_model_public(history_public, cnn_model)

In [ ]:
eval_model_private(history_private, cnn_model)